---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import re
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)
from collections import defaultdict

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    with open('university_towns.txt') as file:
        data = file.read()
        pattern = re.compile(r'([A-Za-z,"]+)\s?\(.*')
        pattern2 = re.compile(r'([A-Za-z\s]+)\[edit\]')
        matches = pattern.findall(data)
        subbed = pattern.sub(r'\1', data)
        #print(len(subbed))
        state = str()
        location_list = list()
        for location in subbed.split('\n'):
            if re.search(pattern2, location):
                state = re.search(pattern2, location).group(1)
            else:
                if (location):
                    location_list.append((state,location))
                    #location_dict[state].append(location)
                #print(pattern2.sub(r'\1', location))
        #print(len(location_list))
        df = pd.DataFrame.from_records(location_list, columns = ['State', 'RegionName'])
        return(df)
#print(get_list_of_university_towns())

In [5]:
def is_recession(row):
    #data = row[['growth', 'shifted_growth']]
    if row['growth'] > 0 and row['shifted_growth'] > 0:
        return True
    if row['growth'] < 0 and row['shifted_growth'] < 0:
        return False
    return np.nan

In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', skiprows = 7)
    gdp.columns = ['Annual','Annual GDP in billions','Annual GDP in billions of chained 2009 dollars','Unnamed: 3','Quarter',
                                                 'Quarter GDP in billions of current dollars','Quarter GDP in billions of chained 2009 dollars','Timestamp']
    gdp.drop(['Unnamed: 3', 'Timestamp'], axis = 1)
    gdp_annual = gdp[['Annual','Annual GDP in billions','Annual GDP in billions of chained 2009 dollars']]
    gdp_quarter = gdp[['Quarter','Quarter GDP in billions of current dollars','Quarter GDP in billions of chained 2009 dollars']]
    gdp_quarter.drop(gdp_quarter.index[:212], inplace = True)#[:212]
    gdp_quarter['growth'] = gdp_quarter['Quarter GDP in billions of chained 2009 dollars'].diff(-1) * (-1)
    gdp_quarter['shifted_growth'] = gdp_quarter['growth'].shift(1)
    gdp_quarter['recession'] = gdp_quarter.apply(is_recession, axis = 1)
    cols = ['recession']
    gdp_quarter.loc[:,cols] = gdp_quarter.loc[:,cols].ffill().fillna(True)
    recession_start_index = int(gdp_quarter[gdp_quarter['recession'] == False].index[0])
    #gdp_quarter[gdp_quarter['recession'] == False].index
    return (gdp_quarter.loc[recession_start_index, 'Quarter'])

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', skiprows = 7)
    gdp.columns = ['Annual','Annual GDP in billions','Annual GDP in billions of chained 2009 dollars','Unnamed: 3','Quarter',
                                                 'Quarter GDP in billions of current dollars','Quarter GDP in billions of chained 2009 dollars','Timestamp']
    gdp.drop(['Unnamed: 3', 'Timestamp'], axis = 1)
    gdp_annual = gdp[['Annual','Annual GDP in billions','Annual GDP in billions of chained 2009 dollars']]
    gdp_quarter = gdp[['Quarter','Quarter GDP in billions of current dollars','Quarter GDP in billions of chained 2009 dollars']]
    gdp_quarter.drop(gdp_quarter.index[:212], inplace = True)#[:212]
    gdp_quarter['growth'] = gdp_quarter['Quarter GDP in billions of chained 2009 dollars'].diff(-1) * (-1)
    gdp_quarter['shifted_growth'] = gdp_quarter['growth'].shift(1)
    gdp_quarter['recession'] = gdp_quarter.apply(is_recession, axis = 1)
    cols = ['recession']
    gdp_quarter.loc[:,cols] = gdp_quarter.loc[:,cols].ffill().fillna(True)
    recession_start_index = int(gdp_quarter[(gdp_quarter['recession'] == True) & (gdp_quarter['recession'].shift(1) == False)].index[0]) + 1
    #gdp_quarter[gdp_quarter['recession'] == False].index
    return (str(gdp_quarter.loc[recession_start_index, 'Quarter']))
#get_recession_end()

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    return ('2009q2')
#get_recession_bottom()

In [9]:
def get_quarters_map():
    quarters = defaultdict(list)
    for yy in range(0, 17):
        year = yy + 2000
        #print(str(year))
        for q in range(1, 5):
            quarter = str(year) + 'q' +str(q)
            #print( )
            months = [str(year)+'-0'+str(i) if i < 10 else str(year)+'-'+str(i) for i in range(3*(q-1)+1 ,3*q+1)]
            quarters[quarter] = months
    return(quarters)

In [10]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_data.drop(['1996-04', '1996-05', '1996-06', '1996-07', '1996-08', '1996-09', '1996-10', '1996-11', '1996-12',
                       '1997-01', '1997-02', '1997-03', '1997-04', '1997-05', '1997-06', '1997-07', 
                       '1997-08', '1997-09', 'RegionID', 'Metro', 'CountyName', 'SizeRank',
                       '1997-10', '1997-11', '1997-12', '1998-01', '1998-02', '1998-03', '1998-04', '1998-05', '1998-06', 
                       '1998-07', '1998-08', '1998-09', '1998-10', '1998-11', '1998-12', '1999-01', '1999-02', '1999-03', 
                       '1999-04', '1999-05', '1999-06', '1999-07', '1999-08', '1999-09', '1999-10', '1999-11', '1999-12'],axis = 1,inplace = True)
    #print(list(housing_data.columns))
    housing_data['State'] = housing_data['State'].map(states)

    quarter_converter = get_quarters_map()
    for quarter, months in quarter_converter.items():
        #print(quarter, months)
        if quarter not in ['2016q3','2016q4']:
            housing_data[quarter] = housing_data[months].mean(axis = 1)
            housing_data.drop(months,axis = 1,inplace = True)
    housing_data['2016q3'] = housing_data[['2016-07', '2016-08']].mean(axis = 1)
    housing_data.drop(['2016-07', '2016-08'],axis = 1,inplace = True)
    housing_data.set_index(['State','RegionName'], inplace = True)
    return (housing_data)

In [11]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    #print(recession_start, recession_bottom)
    housing_data = convert_housing_data_to_quarters()
    housing_data = housing_data[[recession_start,recession_bottom]]
    housing_data['growth'] = housing_data[recession_bottom] - housing_data[recession_start]
    housing_data.drop([recession_start,recession_bottom],axis = 1,inplace = True)
    #housing_data
    university_towns = get_list_of_university_towns()
    university_towns["data_true"] = True
    all_data = housing_data.merge(university_towns, how='left', left_index=True, right_on = ["State","RegionName"])
    non_university_data = all_data[all_data["data_true"] != True]
    university_data = all_data[all_data["data_true"] == True]
    non_university_data.drop("data_true", axis = 1, inplace = True)
    university_data.drop("data_true", axis = 1, inplace = True)
    ttest_result = stats.ttest_ind(university_data['growth'].dropna(), non_university_data['growth'].dropna())
    p_value = ttest_result[1]
    university_mean = university_data['growth'].mean()
    non_university_mean = non_university_data['growth'].mean()
    if university_mean > non_university_mean:
        better = "university town"
    else:
        better = "non-university town"
    if p_value < 0.01:
        diff = True
    else:
        diff = False
    return(diff,p_value, better)
        
run_ttest()

(True, 0.0043252148535112009, 'university town')